In [2]:
import numpy as np
import skimage
import utils
import pathlib

In [3]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a threshold value

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
        return:
            (int) the computed thresholding value
    """
    assert im.dtype == np.uint8
    ### START YOUR CODE HERE ### (You can change anything inside this block) 
    # You can also define other helper functions
    # Compute normalized histogram
    
    hist, bins = np.histogram(im.ravel(), bins=256, range=(0, 256))
    total_pixels = im.size
    probabilities = hist / total_pixels

    # 2. Initialize variables
    max_between_class_variance = 0
    optimal_threshold = 0

    sum_all_intensity = np.dot(np.arange(256), probabilities)  # Total weighted sum of intensities
    sum_background = 0  # Weighted sum for the background class
    weight_background = 0  # Class 1 weight
    weight_foreground = 0  # Class 2 weight

    # 3. Iterate over all possible thresholds
    for t in range(256):
        # Update background weights and sums
        weight_background += probabilities[t]
        if weight_background == 0:
            continue
        
        weight_foreground = 1 - weight_background
        if weight_foreground == 0:
            break

        sum_background += t * probabilities[t]
        mean_background = sum_background / weight_background
        mean_foreground = (sum_all_intensity - sum_background) / weight_foreground

        # Compute between-class variance
        between_class_variance = (
            weight_background * weight_foreground *
            (mean_background - mean_foreground) ** 2
        )

        # Update optimal threshold if a better one is found
        if between_class_variance > max_between_class_variance:
            max_between_class_variance = between_class_variance
            optimal_threshold = t

    return optimal_threshold
    ### END YOUR CODE HERE ### 

In [4]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("rice-shaded.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

        


Reading image: images/thumbprint.png
Found optimal threshold: 153
Saving image to: image_processed/thumbprint-segmented.png
Reading image: images/rice-shaded.png
Found optimal threshold: 134
Saving image to: image_processed/rice-shaded-segmented.png
